In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import keras as keras
import talos as talos

import time
import sys
import math
%matplotlib inline

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
sys.path.insert(1, '/home/fbegiello/Workspace/Modules')

from activationFunctions import xSigmoid, maxSigmoid, cosDiff, minSin, quadTg, quadTgCapped, maxTanh, sincPlus, quadSinh, quadSinhCapped
from activationFunctions import getCustomActivations
getCustomActivations()

from kerasLogger import learningLogger

W0827 13:39:35.572697 139960123053888 deprecation_wrapper.py:119] From /home/fbegiello/anaconda3/envs/tf/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [ ]:
num_classes = 10
img_rows, img_cols = 28, 28

(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
def mnistModel(x_train, y_train, x_test, y_test, params):

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation=params['activ_1'],
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation=params['activ_1']))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    
    out = model.fit(x_train, y_train,
              batch_size=32,
              epochs=10,
              verbose=0,
              validation_data=(x_test, y_test))
    
    return out, model

In [ ]:
num_classes = 10

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
def mnistModelDense(x_train, y_train, x_test, y_test, params):

    model = Sequential()
    model.add(Dense(512, activation=params['activ_1'], input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation=params['activ_1']))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))

    
    model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])
    
    out = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=0,
                    validation_data=(x_test, y_test))
    
    return out, model

In [3]:
def convertToMatrix(data, step):
    X, Y =[], []
    for i in range(len(data)-step):
        d=i+step  
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

step = 4
N = 1000    
Tp = 800    

t=np.arange(0,N)
x=np.sin(0.02*t)+2*np.random.rand(N)
df = pd.DataFrame(x)

values=df.values
train,test = values[0:Tp,:], values[Tp:N,:]

test = np.append(test,np.repeat(test[-1,],step))
train = np.append(train,np.repeat(train[-1,],step))
 
trainX,trainY =convertToMatrix(train,step)
testX,testY =convertToMatrix(test,step)
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [4]:
def sinModel(x_train, y_train, x_test, y_test, params):

    model = Sequential()
    model.add(SimpleRNN(units=32, input_shape=(1,step), activation=params['activ_1']))
    model.add(Dense(8, activation="relu")) 
    model.add(Dense(1))
    
    model.compile(loss='mean_squared_error',
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])
    
    out = model.fit(trainX, trainY,
                    batch_size=16,
                    epochs=100,
                    verbose=0,
                    validation_data=(testX, testY))
    
    return out, model

In [5]:
from keras.activations import relu, tanh, sigmoid

mnist_activ = {
    'activ_1': [relu, tanh, sigmoid, xSigmoid, maxSigmoid, cosDiff, minSin, maxTanh, sincPlus, quadTg, quadTgCapped, quadSinh, quadSinhCapped],
}

In [6]:
talos.Scan(trainX, trainY, 
           params = mnist_activ, 
           dataset_name='activations_all_rnn',
           model = sinModel)

  0%|          | 0/13 [00:00<?, ?it/s]W0827 13:39:42.630776 139960123053888 deprecation_wrapper.py:119] From /home/fbegiello/anaconda3/envs/tf/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0827 13:39:42.637297 139960123053888 deprecation_wrapper.py:119] From /home/fbegiello/anaconda3/envs/tf/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0827 13:39:42.741794 139960123053888 deprecation_wrapper.py:119] From /home/fbegiello/anaconda3/envs/tf/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0827 13:39:42.985000 139960123053888 deprecation_wrapper.py:119] From /home/fbegiello/anaconda3/envs/tf/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add